In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

from __future__ import print_function
import torch
import torch.nn as nn
import os
import numpy as np

from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from scipy.io import mmread

import pandas as pd


In [2]:
import rec 
# reload(rec)
def pred_recommend(train, test, predicted):
    r = rec.Rec(topk = 5)
    r.fit(train.T)
    t = predicted.T
    targets = np.unique(test.nonzero()[0])
    r.predict( t,targets)
    perf = r.evaluate(test)
    return perf

In [4]:
from cf2 import IBCF
def cf_on_hidden_layer(knn, train, hidden_layers, test):
    test = test
    targets = np.unique(test.nonzero()[0])
    cf = IBCF(sim='cosine',knn=knn,topk=5)
# #     precision, recall = cf.score(train.T, test)
# #     print(precision,recall)
    cf.fit(train.T, X=hidden_layers.T)
    cf.predict(targets)
    perf = (knn,cf.evaluate(test) )
    return perf

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.encoder = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.decoder = nn.Linear(hidden_size, output_size)
        self.repres = dict()

    def forward(self, x):
        out = self.encoder(x)
        self.repres['hidden'] = out.clone()
        out = self.relu(out)
        self.repres['hidden_relu'] = out.clone()
        out = self.decoder(out)
        self.repres['output'] = out.clone()
        return out


In [ ]:
input_size = 6040
hidden_size = 1000
output_size = 6040
loss_history = list()
learning_rate = 0.001

knn = 500

pred_perf = dict()
cf_perf = dict()

net = Net(input_size, hidden_size, output_size)

certeria = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
print(net.train())
num_epochs = 20
for epoch in range(num_epochs):
    for i_batch, sample_batched in enumerate(data.train_loader):
#         print(i_batch,sample_batched)
        sample = Variable(sample_batched['sample'])
#         print(sample)
#         sample = Variable()
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(sample)
        loss = certeria(outputs, sample)
#         print(i_batch, loss.data[0])
        loss.backward()
        optimizer.step()
        
        print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   % (epoch + 1, num_epochs, i_batch + 1, 
                      len(data.train_loader) ,loss.data[0]))
        loss_history.append(loss.data[0])
    repres = dict()
    if (epoch+1) % 5 == 0:
#         model = copy.deepcopy(net)
        name = 'test-repres-%s-epoch_%s_cv.%s'%(str(net).replace('\n',''), epoch+1, cv)
        torch.save(net.state_dict(), name)
        repres[epoch] = dict()
        print('recommendataion on prediction... Epoch %s'%(epoch+1))
        loss_df= pd.DataFrame(loss_history,columns=['loss'])
        loss_df.plot(title='cv = %s, epoch = %s'%(cv,epoch+1))
        predicted = np.zeros_like(data.train_dataset.train)
        # dataset.test_dataset.part = 'all'
        for key in net.repres.keys():
            repres[epoch][key] = np.zeros([net.repres[key].data.numpy().shape[1], data.train_dataset.train.shape[1]])
        for item_id in xrange(data.train_dataset.train.shape[1]):
            sample = Variable(torch.from_numpy(data.train_dataset.train[:,item_id]))
            output = net(sample)
            for key in net.repres.keys():
                r = repres[epoch][key]
                r[:,item_id] = net.repres[key].data.numpy()

        #     print(type(test_input['id']))
            predicted[:,item_id] = output.data.numpy()
        pred_perf[epoch+1] = pred_recommend(train=data.train_dataset.train , test=data.test_dataset.test, predicted= predicted)
        
        for key in net.repres.keys():
            print('recommendataion on cf_hidden... Epoch %s, layper %s'%(epoch, key))
            hidden_layer = output.data.numpy()
            cf_result = cf_on_hidden_layer(knn=knn,train = data.train_dataset.train, hidden_layers=repres[epoch][key], test=data.test_dataset.test)
            cf_perf[(epoch+1, key)] = (cf_result)            
